In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

import lightgbm as lgb
import xgboost as xgb
import sklearn.datasets
import gc

In [2]:
# See Gradient Boosting for date conversion scripts

# train_X = pd.read_csv('music_data/train_X.csv')
train_X = pd.read_csv('music_data/train_X_date_converted.csv')
train_y = pd.read_csv('music_data/train_Y.csv')

# val_X = pd.read_csv('music_data/valid_X.csv')
val_X = pd.read_csv('music_data/valid_X_date_converted.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

# test_X = pd.read_csv('music_data/test_X.csv')
test_X = pd.read_csv('music_data/test_X_date_converted.csv')
test_y = pd.read_csv('music_data/test_Y.csv')

# train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
train_tiny_X = pd.read_csv('music_data/train_tiny_X_date_converted.csv')
train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')

# val_tiny_X = val_X[:1000].copy()
val_tiny_X = pd.read_csv('music_data/val_tiny_X_date_converted.csv')
val_tiny_y = val_y[:1000].copy()

## Tree

### Decision Tree

In [15]:
def train_decision_tree(X, y, n_iter=100, cv=3, scoring='roc_auc'):
    pipelines = {
        "dtclass": make_pipeline(DecisionTreeClassifier(random_state=0))
    }
    decisiontree_hyperparameters = {
        'decisiontreeclassifier__max_depth' : np.arange(3, 10),
        'decisiontreeclassifier__max_features' : np.arange(3, 8),
        'decisiontreeclassifier__min_samples_split' : np.arange(2, 15),
        "decisiontreeclassifier__min_samples_leaf" : np.arange(1,3)
    }
    dtclass_model = RandomizedSearchCV(
        pipelines['dtclass'], decisiontree_hyperparameters, 
        n_iter=n_iter, cv=cv, scoring=scoring, verbose=1
    )
    dtclass_model.fit(X, y['target'])
    return dtclass_model

#### Testing on tiny dataset

In [16]:
dtclass_model_tiny = train_decision_tree(train_tiny_X, train_tiny_y)
print(dtclass_model_tiny.best_params_)
print(classification_report(val_tiny_y['target'], dtclass_model_tiny.predict(val_tiny_X)))
print(roc_auc_score(val_tiny_y['target'], dtclass_model_tiny.predict(val_tiny_X)))

SystemError: Bad call flags in PyCFunction_Call. METH_OLDARGS is no longer supported!

#### Running real data

In [ ]:
dtclass_model = train_decision_tree(train_X, train_y)

In [ ]:
print(dtclass_model.best_params_)
print(classification_report(val_y['target'], dtclass_model.predict(val_X)))
print(roc_auc_score(val_y['target'], dtclass_model.predict(val_X)))

### Random Forest